In [1]:
import sympy as sym
import sympy.physics.mechanics as me
from sympy.abc import t as time
import pandas as pd
import seaborn as sns

import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

import sys, os

sys.path.insert(1, os.path.join(sys.path[0], '../..'))
import custom_plot_objects as cpo

import ModelFramework as mf
import ModelFramework.Elements as ele
import ModelFramework.ExternalForces as ef
import FwtModels.RectWing as rw
import FwtModels.RollRig as RollRig

from matplotlib.lines import Line2D

me.mechanics_printing()

In [20]:
half = sym.Rational(1,2)

p = RollRig.base_params(1)
p.m_w[0,0]

TypeError: 'ModelSymbol' object is not subscriptable

In [30]:
# Generate Referecne Frame
wing_frame = mf.HomogenousTransform().R_x(p.q[0])#.R_y(p.alpha_r)

#Generate Mass Matrices
M_w = ele.MassMatrix(p.m_w,I_xx = sym.Rational(1,12)*0.305*p.s**3)


# Generate Rigid Elements
bar_wing = ele.RigidElement(wing_frame,M_w)

In [31]:
# Main Wing Aero Forces 
wing_AeroForces = ef.AeroForce.PerUnitSpan(p,wing_frame.Translate(0,p.y_w,0),p.a_0,
                               alphadot = 0,
                               M_thetadot = 0,
                               e = 0,
                               w_g = p.w_g,
                               rootAlpha = p.alpha_r,
                               alpha_zero = 0,
                               stall_angle = 0,
                               c = p.c,
                               c_d_max = 0,
                               linear = True).integrate((p.y_w,-p.s*half,p.s*half))


In [32]:
CompositeForce = ef.CompositeForce([wing_AeroForces])
# create instance of the model
sm = mf.SymbolicModel.FromElementsAndForces(p,[bar_wing],ExtForces = CompositeForce)

In [33]:
sm.M

⎡                    3⎤
⎣0.0254166666666667⋅s ⎦

In [34]:
eom = (sm.M.inv()*(-sm.f+sm.ExtForces.Q()))[0]+p.beta*sym.Rational(1,2)*p.rho*p.V**2*p.c*0.12*2*sym.pi
eom

In [35]:
x = []
eom = (sm.subs(p.GetSubs(0,None,ignore=[p.V,p.s])).M.inv()*(-sm.f+sm.ExtForces.Q()))[0]+p.beta*sym.Rational(1,2)*p.rho*p.V**2*p.c*0.12*2*sym.pi

eom = eom.subs({p.c:0.067,p.a_0:2*sym.pi,p.rho:1.225})
eom
-sm.f+sm.ExtForces.Q()

⎡           3    ⎤
⎢-V⋅a₀⋅c⋅ρ⋅s ⋅q₀̇ ⎥
⎢────────────────⎥
⎣       24       ⎦

In [36]:
args = eom.args
func_beta = sym.lambdify((p.V,p.s,p.qd[0]),-args[1]/args[0]*p.beta)
func_beta(15,1,np.deg2rad(141.4))

In [37]:
vs = [15,17.5,20,22.5,25,27.5,30]
rate = [141.4,168.6,195.2,224.1,258.6,290.1,319.9]
beta = [func_beta(vs[i],1,np.deg2rad(rate[i])) for i,v in enumerate(vs)]
print(beta)
np.mean(beta)



[4.4952560333697305, 4.5942624806595465, 4.654211338651545, 4.749584521820635, 4.932701033505286, 5.030501897700497, 5.084980215965264]


In [38]:
func_qd = sym.lambdify((p.V,p.s,p.beta),args[0]/-args[1]*p.qd[0])
rate= [func_qd(v,0.72,np.mean(beta))/np.pi*180 for v in vs]
rate

In [18]:
sm.M.inv()

⎡39.344262295082⎤
⎢───────────────⎥
⎢        3      ⎥
⎣       s       ⎦